In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%matplotlib widget

In [4]:
from trsm import TRSM, combos

In [5]:
import awkward as ak

In [6]:
filename = f'../../signal/skimmed/NMSSM_XYH_YToHH_6b_MX_700_MY_400_testing_set_skimmed.root'
trsm = TRSM(filename=filename)

-- [INFO] -- /uscms/home/srosenzw/nobackup/miniconda3/envs/work/lib/python3.8/runpy.py -- Opening ROOT file ../../signal/skimmed/NMSSM_XYH_YToHH_6b_MX_700_MY_400_testing_set_skimmed.root with columns
----------------------------------------------------------------------------------------------------
                                            TABLE COLUMNS                                           
----------------------------------------------------------------------------------------------------
-- [INFO] -- /uscms/home/srosenzw/nobackup/miniconda3/envs/work/lib/python3.8/runpy.py -- Tree contains 269883 events.
jet_pt                            jet_eta                           jet_phi                           
jet_m                             jet_btag                          jet_qgl                           
jet_idx                           jet_hadronFlav                    jet_partonFlav                    
HX_b1_recojet_m                   HX_b1_recojet_pt                  H

In [7]:
combos7 = combos(trsm, 7)

100%|█████████████████████████████████████████████████████| 269883/269883 [01:42<00:00, 2637.16it/s]


Total events chosen: 40778


In [8]:
combos7.HX_b1_p4[combos7.combo_mask]

<MomentumArray4D [{pt: 158, eta: -0.179, ... m: 13.1}] type='40778 * Momentum4D[...'>

In [9]:
HX_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 0))
HX_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 1))
H1_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 2))
H1_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 3))
H2_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 4))
H2_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 5))

In [10]:
from keras.models import model_from_json
from pickle import load

In [11]:
tag = '2HL_40_20'

In [12]:
json_file = open(f'../models/{tag}/model/model_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(f'../models/{tag}/model/model_1.h5')
scaler = load(open(f'../models/{tag}/model/scaler_1.pkl', 'rb'))

2021-07-09 10:22:22.723863: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
test_features_7 = scaler.transform(combos7.combo_features)
scores_7 = loaded_model.predict(test_features_7)[:,0]
scores_7.shape

2021-07-09 10:22:23.660769: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-09 10:22:23.661188: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2000000000 Hz


(285446,)

In [14]:
sgnl_scores_7 = scores_7[combos7.sgnl_mask]
bkgd_scores_7 = scores_7[~combos7.sgnl_mask]

In [15]:
import awkward as ak
import numpy as np

In [16]:
combo_mask = combos7.combo_mask
HX_p4 = ak.to_numpy(combos7.HX_p4.m)[combo_mask]
H1_p4 = ak.to_numpy(combos7.H1_p4.m)[combo_mask]
H2_p4 = ak.to_numpy(combos7.H2_p4.m)[combo_mask]

correct_H = np.concatenate((HX_p4, H1_p4, H2_p4))
incorrect_H = ak.to_numpy(ak.flatten(combos7.incorrect_H_p4.m))

correct_H = correct_H.reshape(correct_H.shape[0],)
incorrect_H = incorrect_H.reshape(incorrect_H.shape[0],)

In [17]:
combos7.combo_features[0]

array([148.5       , 131.        ,  98.        ,  94.625     ,
        71.        ,  61.40625   ,  -0.17868042,  -0.50427246,
        -0.38952637,  -0.34362793,   0.35101318,   0.74560547,
         1.58886719,  -1.23364258,  -2.36816406,   2.56542969,
        -0.63183594,   0.74401855,   0.99072266,   0.55615234,
         0.97900391,   0.99951172,   0.99902344,   0.17663574,
       150.31040491, 128.84674912,  97.88692781,  96.12824239,
        69.91360971,  61.471839  ])

In [18]:
import matplotlib.pyplot as plt
from consistent_plots import hist, hist2d

In [19]:
def norm_hist(arr, bins=100):
    n, b = np.histogram(arr, bins=bins)
    x = (b[:-1] + b[1:]) / 2
    
    return n/n.max(), b, x

In [20]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_1, b7, x7 = norm_hist(incorrect_H, bins=np.linspace(0,1000,100))
n7_2, b7, x7 = norm_hist(correct_H, bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Incorrect')
hist(ax, x7, weights=n7_2, bins=b7, label='Correct')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[HX_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[HX_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(HX_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='HX b1')
hist(ax, x7, weights=n7_1, bins=b7, label='HX b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True HX')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations Non-Normalized")

n, *_ = hist(ax, incorrect_H[HX_b1_mask], bins=b7, label='HX b1')
n, *_ = hist(ax, incorrect_H[HX_b2_mask], bins=b7, label='HX b2')
n, *_ = hist(ax, HX_p4, bins=b7, label='True HX')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('Count')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[H1_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[H1_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(H1_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H1 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H1 b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True H1')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')


ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations Non-Normalized")

n, *_ = hist(ax, incorrect_H[H1_b1_mask], bins=b7, label='H1 b1')
n, *_ = hist(ax, incorrect_H[H1_b2_mask], bins=b7, label='H1 b2')
n, *_ = hist(ax, H1_p4, bins=b7, label='True HX')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('Count')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,4))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[H2_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[H2_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(H2_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H2 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H2 b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations Non-Normalized")

hist(ax, incorrect_H[H2_b1_mask], bins=b7, label='H2 b1')
hist(ax, incorrect_H[H2_b2_mask], bins=b7, label='H2 b2')
hist(ax, H2_p4, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('Count')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_1, b7, x7 = norm_hist(incorrect_H[scores_7[~combos7.sgnl_mask] <= 0.8], bins=np.linspace(0,1000,100))
n7_2, b7, x7 = norm_hist(incorrect_H[scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Low score')
hist(ax, x7, weights=n7_2, bins=b7, label='High Score')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Incorrect Higgs mass')
ax.set_ylabel('AU')

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations")

n7_1, b7 = np.histogram(incorrect_H[scores_7[~combos7.sgnl_mask] <= 0.8], bins=np.linspace(0,1000,100))
n7_2, b7 = np.histogram(incorrect_H[scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Low score')
hist(ax, x7, weights=n7_2, bins=b7, label='High Score')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Incorrect Higgs mass')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig, axs = plt.subplots(nrows=2, ncols=2, figsize=(10,10))

ax = axs[0][0]
n, x, y, im = hist2d(ax, incorrect_H, scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

ax = axs[0][1]
n, x, y, im = hist2d(ax, incorrect_H[HX_b1_mask | HX_b2_mask], scores_7[~combos7.sgnl_mask][HX_b1_mask | HX_b2_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

ax = axs[1][0]
n, x, y, im = hist2d(ax, incorrect_H[H1_b1_mask | H1_b2_mask], scores_7[~combos7.sgnl_mask][H1_b1_mask | H1_b2_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

ax = axs[1][1]
n, x, y, im = hist2d(ax, incorrect_H[H2_b1_mask | H2_b2_mask], scores_7[~combos7.sgnl_mask][H2_b1_mask | H2_b2_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)
ax.set_ylabel("Assigned Score")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
fig, ax = plt.subplots(nrows=2, ncols=3)

n, x, y, im = hist2d(ax, incorrect_H, scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)

ax.set_ylabel("Assigned Score")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

AttributeError: 'numpy.ndarray' object has no attribute 'hist2d'

In [ ]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[HX_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[HX_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='HX b1')
hist(ax, x7, weights=n7_1, bins=b7, label='HX b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[H1_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[H1_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H1 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H1 b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[H2_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[H2_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H2 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H2 b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()